In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

REBUILD_DATA = False #set to true to once, then back to false unless you want to change something in your training data

class CovidvsNormal():
    IMG_SIZE = 80
    NORMAL = "NORMAL"
    COVID = "COVID-19"
    LABELS = {NORMAL:0, COVID:1}
    
    training_data = []
    normalcount = 0
    covidcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "png" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                        if label == self.NORMAL:
                            self.normalcount += 1
                        elif label == self.COVID:
                            self.covidcount += 1
                    except Exception as e:
                        pass
                
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("NORMAL:", self.normalcount)
        print("COVID:", self.covidcount)


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #input image, 32 convolutional features, 5*5 kernel
        self.conv2 = nn.Conv2d(32, 64, 5 )
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            print(self._to_linear)
        return x
        
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")    
    
net = Net().to(device)

if REBUILD_DATA:
    covidvnormal = CovidvsNormal()
    covidvnormal.make_training_data()

training_data = np.load("training_data.npy")
print(len(training_data))

running on the GPU
torch.Size([128, 2, 2])
512
NORMAL


100%|██████████████████████████████████████████████████████████████████████████████| 1341/1341 [00:34<00:00, 38.67it/s]


COVID-19


100%|████████████████████████████████████████████████████████████████████████████████| 219/219 [00:07<00:00, 29.51it/s]


NORMAL: 1341
COVID: 219
1560


In [3]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0 
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

156
1404
156


In [4]:
def fwd_pass(X, y, train = False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches =  [torch.argmax[i] == torch.argmax(j) for i,j in zip(outputs,y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss     